##### chunk train videos csv

In [ ]:
import pandas as pd
import cv2
datalist_path='/tsukimi/datasets/Chiba/baseline/datalist_3/train.csv'
# read datalist
datalist = pd.read_csv(datalist_path,header=None, delimiter=',')
labels=list(datalist.values[:,1])
videos=list(datalist.values[:,0])

def process_video(video_path,label,segment_length = 17):
    # read video
    cap = cv2.VideoCapture(video_path)
    # print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
    video_segments = []
    
    # clip video to 17 frames each
    frame_count = 0
    frames=[]
    while True:
        success = True
        if not frames:
            # read first frame
            success, frame = cap.read()
            if not success:
                break
            frames.append(frame)

        for _ in range(segment_length-1):
            success, frame = cap.read()
            if not success:
                break 
            frames.append(frame)
            
        # Check if we have collected 17 frames
        if len(frames) == segment_length:
            start_frame = frame_count
            end_frame = frame_count + segment_length - 1
            segment_info = {
                'video_path': video_path,
                'label': label,
                'start_frame': start_frame,
                'end_frame': end_frame
            }
            video_segments.append(segment_info)
            # reset frames, last frame will be used as first frame in next segment
            frames = [frames[-1]]
            frame_count=end_frame
        else:
            break
    cap.release()
    df = pd.DataFrame(video_segments)
    return df


In [ ]:
data=process_video(videos[0],labels[0])
for video, label in zip(videos[1:], labels[1:]):
    df = process_video(video,label)
    data = pd.concat([data, df], ignore_index=True)
print(data)

In [ ]:
# save to '/tsukimi/datasets/Chiba/baseline/datalist_3/train_16.csv'
data.to_csv('/tsukimi/datasets/Chiba/baseline/datalist_3/train_16.csv', index=False)

In [ ]:
# count label distribution
labels=data['label'].apply(lambda x:x.split('&'))
labels

In [ ]:
dist={}
for label in labels:
    for l in label:
        if l not in dist:
            dist[l]=1
        else:
            dist[l]+=1
print(dist)

In [ ]:
# check if empty row exists
data.isnull().sum()